<img src="https://i.imgur.com/gb6B4ig.png" width="400" alt="Weights & Biases" />

# Что такое W&B?

`Weights & Biases (W&B)` - это многофункциональная платформа для отслеживания экспериментов, логирования результатов, которая позволяет облегчить и ускорить рутинные процессы при обучении моделей.

# Ключевые особенности

* Позволяет отслеживать показатели работы модели в режиме реального времени и сразу же выявляйте проблемные места.
* Визуализация результатов - графики, изображения, видео, аудио, 3D-объекты и многое [другое](https://docs.wandb.ai/guides/track/log#logging-objects).
* Позволяет тренировать модели с разных устройств и хранить результаты в одном месте, так же удобно при командной работе над задачей
* Использование артефактов W&B позволяет отслеживать и создавать версии ваших наборов данных, моделей, зависимостей и результатов в конвейерах машинного обучения.
* Низкий порог входа (можно начать с 6 строк кода), так же есть готовые интеграции со всеми популярными DS фреймворками

![centralized dashboard](https://i.imgur.com/BGgfZj3.png)

## Установка `wandb`

`wandb` (библиотека W&B), по умолчанию установлена на Kagglе

Поскольку образ kaggle не часто обновляется, а `wandb` постоянно выпускает новые версии,рекомендуется установить последнюю версию с помощью флага `--upgrade`.

In [1]:
!pip install --upgrade -q wandb

## Регистрация учетной записи и вход в систему

Вам понадобится уникальный ключ API для входа в Weights & Biases.

1. Если у вас нет учетной записи Weights & Biases, вы можете перейти на https://wandb.ai/site и создать БЕСПЛАТНУЮ учетную запись.
2. Доступ к ключу API: https://wandb.ai/authorize.

На Kaggle можно авторизоваться в W&B двумя способами:

1. С помощью `wandb.login ()`. Он запросит ключ API, который вы можете скопировать + вставить.
2. Используя Kaggle secrets для хранения ключа API и использовать приведенный ниже фрагмент кода для входа в систему. Прочтите это [обсуждение](https://www.kaggle.com/product-feedback/114053), чтобы узнать больше о Kaggle secrets.

```python
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("wandb_api") 
wandb.login(key=wandb_api)
```
[Подробнее о входе в W&B](https://docs.wandb.ai/ref/cli/wandb-login).

In [2]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ivanich. Use `wandb login --relogin` to force relogin


True

## Возьмем, для примера, реализацию `MLP` из раздела про нейросети

In [3]:
import torch
import os
import copy
import sys
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
# Загружаем файлы датасета

car_info = pd.read_csv('../data/car_info.csv')   # car_info - информация про машины с таргетом
 
rides_info = pd.read_csv('../data/rides_info.csv') # rides_info - информация про поездки

In [5]:
rides_info = rides_info.merge(car_info, on = 'car_id', how = 'left')

In [6]:
drop_cols = ['user_id', 'car_id', 'ride_id', 'ride_date']
cat_cols = ['car_type', 'fuel_type', 'model']

In [7]:
# закодируем категориальные фичи в one hot encoding вектора
rides_info = pd.get_dummies(rides_info, columns=cat_cols)

# заполним пропущенные значения медианным значением по столбцу
rides_info.fillna(rides_info.median(), inplace=True)

/tmp/ipykernel_117170/3567080960.py:5: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  rides_info.fillna(rides_info.median(), inplace=True)


In [8]:
rides_df = rides_info.drop(columns=drop_cols)

In [9]:
# переведем строковые значения категориального таргета в целочисленные
le = LabelEncoder()
rides_df['target_class'] = le.fit_transform(rides_df['target_class'])

In [11]:
# числовые переменные, которые подвергнем трансформации
num_cols = [col for col in list(rides_df.columns)
            if col not in ['target_reg', 'target_class']]

In [12]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

df = scaler.fit_transform(rides_info[num_cols])
df = pd.DataFrame(df, columns=num_cols)

target_scaler = RobustScaler()
target = target_scaler.fit_transform(rides_info['target_reg'].values.reshape(-1, 1))

df['target_reg'] = target
df['target_class'] = rides_info['target_class']

In [13]:
# ВАЖНО! - фиксируем воспроизводимость
def seed_everything(seed=42):
    
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [14]:
# для наших данных и размера нейросети подойдет запуск на cpu
device = torch.device('cpu')

### Сохраняем гиперпараметры в виде словаря, для того чтобы позже зарегистрировать этот конфигурационный файл в W&B.

In [15]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [16]:
# Определяем гиперпараметры

CONFIG = dict (
    hidden_size=128,
    dropout=0.1,
    lr=1e-3,
    batch_size=128,
    num_workers=os.cpu_count(),
    epochs=10,
    num_features=train.shape[1]-2, # кол-во фичей подаваемое на вход
    num_tar_2=train.target_class.nunique(), # количество выходов равно кол-ву предсказываемых классов
    architecture = "MLP",
    infra = "Kaggle"
)

##  Строим входной конвейер

In [17]:
# датасет выдает фичи и значения целевых переменных
class Rides(Dataset):
    
    def __init__(self, df):
        self.df = df
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx,:]
        
        data = row.drop(labels=['target_reg', 'target_class'])
        
        data = torch.FloatTensor(data.values.astype('float'))
        tar_1 = torch.tensor(row['target_reg']).float()
        tar_2 = row['target_class'].astype('int')
        
        return data, tar_1, tar_2

In [18]:
# Build data loaders

train_datasets = {'train': Rides(train),
                  'val': Rides(test)}

dataloaders = {x: torch.utils.data.DataLoader(train_datasets[x], 
                                                   batch_size=CONFIG['batch_size'], 
                                                   shuffle=True, 
                                                   num_workers=CONFIG['num_workers'])
                    for x in ['train', 'val']}

## Настраиваем модель

In [19]:
# Построим архитектуру mlp с двумя головами для регрессии и классифкации

class TabularNN(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.mlp = nn.Sequential(
                          nn.Linear(cfg['num_features'], cfg['hidden_size']),
                          nn.Dropout(cfg['dropout']),
                          nn.ReLU(),
                          nn.Linear(cfg['hidden_size'], cfg['hidden_size']),
                          nn.Dropout(cfg['dropout']),
                          nn.ReLU(),
                          nn.Linear(cfg['hidden_size'], cfg['hidden_size'] // 2),
                          )
        
        self.regressor = nn.Sequential(
            nn.Linear(cfg['hidden_size'] // 2, 1)
        )
        self.classifier = nn.Sequential(
            nn.Linear(cfg['hidden_size'] // 2, cfg['num_tar_2'])
        )

    def forward(self, data):
        x = self.mlp(data)
        tar1 = self.regressor(x)
        tar2 = self.classifier(x)
        return tar1.view(-1), tar2

## Обучаем и логируем результаты в W&B

В этом разделе будем использовать:

* [`wandb.init()`](https://docs.wandb.ai/guides/track/launch): инициализировать новый запуск.
* [`wandb.finish()`](https://docs.wandb.ai/ref/python/finish):  завершить и закрыть пробег.
* [`wandb.config`](https://docs.wandb.ai/guides/track/config): объект, в котором хранятся гиперпараметры и настройки, связанные с запуском.


Run (или [объект wandb.Run](https://docs.wandb.ai/ref/python/run)) - это единица вычисления W&B, обычно это 1 эксперимент обучения модели.

In [20]:
# Initialize model
model = TabularNN(CONFIG).to(device)

# оптимайзер и лоссы для регрессии и классификации
optimizer = torch.optim.Adam(model.parameters(), lr = CONFIG['lr'])
regression_criterion = nn.MSELoss().to(device)
classification_criterion = nn.CrossEntropyLoss().to(device)

## Используем `wandb.init ()` для инициализации нового прогона W&B.

В "pipeline" машинного обучения вы можете добавить `wandb.init ()` в начало обучающего сценария, а также сценарий оценки, и каждая часть будет отслеживаться как запуск в W&B.

In [21]:
# Добавим в CONFIG имя модели
CONFIG['model_name'] = 'Tabular_NN'
print('Training configuration: ', CONFIG)

# Initialize W&B run
run = wandb.init(project='Course contest',
                 entity="ivanich",
                 config=CONFIG,
                 group='MLP', 
                 job_type='train')


Training configuration:  {'hidden_size': 128, 'dropout': 0.1, 'lr': 0.001, 'batch_size': 128, 'num_workers': 32, 'epochs': 10, 'num_features': 46, 'num_tar_2': 9, 'architecture': 'MLP', 'infra': 'Kaggle', 'model_name': 'Tabular_NN'}


Ипользуемые аргументы `wandb.init ()`:

* `project`: этот аргумент указывает имя проекта W&B, в который отправляется запуск. Создаем на платформе W&B новый проект с названием «CDSC_DataFeeling_contest» и одновременно отправляем в него run.

* `config`: этот аргумент устанавливает `wandb.config`, объект, подобный словарю, в котором хранятся гиперпараметры, параметры ввода и другие независимые переменные.

* `group`: этот аргумент указывает значение для группировки отдельных прогонов, чтобы было легче сравнивать прогоны из разных архитектур.

* `job_type`: этот аргумент указывает тип выполнения, например, «`train`» или «`evaluate`». Установка типа run упрощает последующую фильтрацию и группировку run, например, чтобы сравнить несколько «`train`» run-ов.

## Обновляем `wandb.config` 

Сохранение конфигурации тренировки полезно для анализа экспериментов и воспроизведения вашей работы позже. С помощью W&B вы также можете группировать run-ы по значениям конфигурации, что означает, что вы можете сравнивать настройки разных run-ов и видеть, как они влияют на результат.

Есть несколько способов настроить `wandb.config`:

* Задать `wandb.config` с аргументом `wandb.init (config)`, как указано выше.
* Установить `wandb.config` напрямую.
* См. Дополнительные параметры настройки в этом [Colab](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/wandb-log/Configs_in_W%26B.ipynb#scrollTo=xFf3zjBSixC1).

In [22]:
# Add "type" and "kaggle_competition" to `wandb.config` directly
wandb.config.type = 'baseline'
wandb.config.kaggle_competition = 'Competitive Data Science Course by Data Feeling'

# Close W&B run
run.finish()

### Проведем серию из 5 экспериментов для подбора параметра `dropout rate`

In [23]:
# Изменим CONFIG - определим dropout как рандомную величину в заданном диапазоне значений
import random

CONFIG['dropout'] = random.uniform(0.01, 0.80)

In [24]:
num_epochs = CONFIG['epochs']
for _ in range(5):
    # 🐝 initialise a wandb run
    wandb.init(project='Course contest',
                 entity="ivanich", 
                config=CONFIG,
                group='MLP', 
                job_type='train'
            )
    
    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100000.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data.
            for inputs, labels_1, labels_2 in dataloaders[phase]:
                inputs = inputs.to(device)
                labels_1 = labels_1.to(device)
                labels_2 = labels_2.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss

                    outputs_1, outputs_2 = model(inputs)
                    loss_1 = regression_criterion(outputs_1, labels_1)
                    loss_2 = classification_criterion(outputs_2, labels_2)

                    loss = loss_1 + loss_2

                    _, preds_2 = torch.max(outputs_2, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                val_acc_history.append(running_loss)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            print('{} Loss: {:.4f}'.format(phase, epoch_loss))
            
            # 🐝 Log train and validation metrics to wandb
            wandb.log({'{} loss'.format(phase): epoch_loss})

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_model_wts = copy.deepcopy(model.state_dict())


    # load best model weights
    model.load_state_dict(best_model_wts)
    
    # 🐝 Close your wandb run 
    wandb.finish()

Epoch 0/9
----------


AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/tmp/ipykernel_117170/4292527571.py", line 17, in __getitem__
    tar_2 = row['target_class'].astype('int')
AttributeError: 'str' object has no attribute 'astype'


*Используйте `run.finish ()`, чтобы закрыть инициализированный run W&B после завершения `job_type`.*

*Каждый run получает свою собственную страницу, на вкладках которой содержится дополнительная информация о запуске.*

## Создание артефакта W&B.

W&B Artifacts позволяет вам регистрировать данные, которые входят (например, набор данных) и выходят (например, веса обученной модели) этих процессов.

Другими словами, артефакты - это способ сохранить ваши наборы данных и модели. Вы можете использовать [этот Colab](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/wandb-artifacts/Pipeline_Versioning_with_W%26B_Artifacts.ipynb), чтобы узнать больше об артефактах.

### Сохраняем свою работу с помощью `wandb.log_artifact ()` 

В run есть три шага для создания и сохранения артефакта модели.

1. Создайте пустой артефакт с помощью `wandb.Artifact ()`.
2. Добавьте файл модели в Артефакт с помощью `wandb.add_file ()`.
3. Вызовите `wandb.log_artifact ()`, чтобы сохранить Артефакт.

In [ ]:
# Save model
torch.save(model.state_dict(), 'tab_model.pth')

# Initialize a new W&B run
run = wandb.init(project='Course contest',
                 entity="ivanich", 
                config=CONFIG,
                group='MLP', 
                 job_type='save') # Note the job_type

# Update `wandb.config`
wandb.config.type = 'baseline'
wandb.config.kaggle_competition = 'Competitive Data Science Course by Data Feeling'

# Save model as Model Artifact
artifact = wandb.Artifact(name='best_tab_NN', type='model') # Задаем произвольное имя
artifact.add_file('tab_model.pth')
run.log_artifact(artifact)

# Finish W&B run
run.finish()

## Теперь залогируем бустинг

In [ ]:
#!pip install xgboost

In [26]:
import xgboost
import json
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split

In [27]:
run = wandb.init(project='Course contest',
                 entity="ivanich",
                 job_type='train-model',
                 group='XGB',
                 config={'wandb_nb':'wandb_course_contest'})  # config is optional here

In [28]:
bst_params = {
         'gamma': 1,               ## def: 0
         'learning_rate': 0.1,     ## def: 0.1
        'max_depth': 3,
        'min_child_weight': 100,  ## def: 1
        'n_estimators': 25,
        'nthread': 4,
        'random_state': 42,
        'reg_alpha': 0,
        'reg_lambda': 0,          ## def: 1
        'eval_metric': [ 'mlogloss'],
        'tree_method': 'hist'  # use `gpu_hist` to train on GPU
    }

In [29]:
run.config.update(dict(bst_params))
run.config.update({'early_stopping_rounds': 40})

In [33]:
X = df.drop(['target_reg','target_class'], axis=1)
y = rides_df['target_class']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

В `wandb` есть встроенная поддержка библиотеки XGBoost через callback

In [39]:
from wandb.lightgbm import wandb_callback, log_summary
import lightgbm as lgb

xgbmodel = lgb.LGBMClassifier(**bst_params, use_label_encoder=False)

# Train the model, using the wandb_callback for logging
xgbmodel.fit(X_train, y_train, eval_set=[(X_test, y_test)], 
             early_stopping_rounds=run.config['early_stopping_rounds'],
             callbacks=[wandb_callback()])

bstr = xgbmodel.booster_
bstr.save_model("lgb.mod")

[1]	valid_0's multi_logloss: 2.12266
Training until validation scores don't improve for 40 rounds
[2]	valid_0's multi_logloss: 2.07981
[3]	valid_0's multi_logloss: 2.0463
[4]	valid_0's multi_logloss: 2.01899
[5]	valid_0's multi_logloss: 1.99613
[6]	valid_0's multi_logloss: 1.97649
[7]	valid_0's multi_logloss: 1.95974
[8]	valid_0's multi_logloss: 1.94527
[9]	valid_0's multi_logloss: 1.9326
[10]	valid_0's multi_logloss: 1.92094
[11]	valid_0's multi_logloss: 1.91083
[12]	valid_0's multi_logloss: 1.9013
[13]	valid_0's multi_logloss: 1.8926
[14]	valid_0's multi_logloss: 1.88489
[15]	valid_0's multi_logloss: 1.87779
[16]	valid_0's multi_logloss: 1.87106
[17]	valid_0's multi_logloss: 1.86467
[18]	valid_0's multi_logloss: 1.85874
[19]	valid_0's multi_logloss: 1.84907
[20]	valid_0's multi_logloss: 1.84341
[21]	valid_0's multi_logloss: 1.83864
[22]	valid_0's multi_logloss: 1.83376
[23]	valid_0's multi_logloss: 1.82731
[24]	valid_0's multi_logloss: 1.82192
[25]	valid_0's multi_logloss: 1.81685
Di

In [ ]:
from wandb.xgboost import wandb_callback

# Initialize the XGBoostClassifier
xgbmodel = xgboost.XGBClassifier(**bst_params, use_label_encoder=False)

# Train the model, using the wandb_callback for logging
xgbmodel.fit(X_train, y_train, eval_set=[(X_test, y_test)], 
             early_stopping_rounds=run.config['early_stopping_rounds'],
             callbacks=[wandb_callback()])

bstr = xgbmodel.get_booster()

In [41]:
# Save the booster to disk
model_name = f'{run.name}_model.json'
bstr.save_model(model_name)

# Get the booster's config
#config = json.loads(bstr.save_config())

# Log the trained model to W&B Artifacts, including the booster's config
model_art = wandb.Artifact(name=model_name, type='model',) #metadata=dict(config))
model_art.add_file(model_name)
run.log_artifact(model_art);

In [43]:
# Log booster metrics
run.summary["best_score"] = bstr.best_score
run.summary["best_iteration"] = bstr.best_iteration
#run.summary["best_ntree_limit"] = bstr.best_ntree_limit

# Log validation metrics
preds = xgbmodel.predict(X_test)
run.summary["f1_score"] = f1_score(y_test, preds, average='macro')
run.summary["accuracy"] = accuracy_score(y_test, preds)

#### Finish the W&B Run

In [44]:
run.finish()

iteration,▁▂▂▃▄▅▅▆▇▇▁▂▂▃▄▅▅▆▇█▁▂▂▃▄▅▆▆▇█▁▂▃▃▄▅▆▆▇█
valid_0_multi_logloss,█▆▅▄▃▃▂▂▁▁█▆▅▄▃▃▂▂▁▁█▆▅▄▃▃▂▂▁▁▇▆▄▄▃▃▂▂▁▁
accuracy,0.32011
best_iteration,25
f1_score,0.31565
iteration,24


# ❄️ Ресурсы

Вот несколько релевантных ссылок по теме:


* Ознакомьтесь с [официальной документацией](https://docs.wandb.ai/), чтобы узнать больше о передовых методах работы и дополнительных функциях.

* Ознакомьтесь с [репозиторием GitHub с примерами](https://github.com/wandb/examples), где вы найдете тщательно отобранные и минимальные примеры. Это может быть хорошей отправной точкой.

Вот некоторые другие Kaggle ноутбуки с использованием Weights & Biases, которые могут быть вам полезны.

* [EfficientNet+Mixup+K-Fold using TF and wandb](https://www.kaggle.com/ayuraj/efficientnet-mixup-k-fold-using-tf-and-wandb)

* [HPA: Segmentation Mask Visualization with W&B](https://www.kaggle.com/ayuraj/hpa-segmentation-mask-visualization-with-w-b)

* [HPA: Multi-Label Classification with TF and W&B](https://www.kaggle.com/ayuraj/hpa-multi-label-classification-with-tf-and-w-b)

* [🐦BirdCLEF: Quick EDA with W&B](https://www.kaggle.com/ayuraj/birdclef-quick-eda-with-w-b)